In [1]:
import torch
from torch import Tensor
from torch.nn import Linear, ReLU, Sequential
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import HeteroConv, GATConv, SAGEConv, Linear
from torch_geometric.nn.aggr import Aggregation, MultiAggregation
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import OptPairTensor, Adj, Size
import os.path as path
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import tqdm
from typing import Optional, Union, List, Tuple

In [2]:
data_folder = "ds/"
perc = 0
year = 2023
month = 11
# test_hd = f"full_hd_nomatch_{perc}.pt"
# test_hd = f"full_hd_{perc}.pt"
test_hd = f"full_hdmb_{perc}.pt"
# train_collab_with_filename = f"collab_with_nomatch_{year}_{month}_{perc}.pt"
# train_collab_with_filename = f"collab_with_{year}_{month}_{perc}.pt"
train_collab_with_filename = f"collab_withmb_{year}_{month}_{perc}.pt"
best_threshold = 0.69
# model_path = f"model_main_nomatch_{year}_{month}_{perc}.pth"
# model_path = f"model_main_{year}_{month}_{perc}.pth"
model_path = f"model_main_mb_{year}_{month}_{perc}.pth"

In [3]:
data = torch.load(path.join(data_folder, test_hd))

data.validate()

/tmp/ipykernel_83469/624821894.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(path.join(data_folder, test_hd))


True

In [4]:
artist_channels = data["artist"].x.size(1)
track_channels = data["track"].x.size(1)
tag_channels = data["tag"].x.size(1)

print(f"Artist channels: {artist_channels}")
print(f"Track channels: {track_channels}")
print(f"Tag channels: {tag_channels}")

Artist channels: 15
Track channels: 4
Tag channels: 24


In [5]:
train_collab_with = torch.load(path.join(data_folder, train_collab_with_filename))
train_edges_set = set(map(tuple, train_collab_with.t().tolist()))

/tmp/ipykernel_83469/3191979287.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_collab_with = torch.load(path.join(data_folder, train_collab_with_filename))


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

Device: 'cuda'


In [7]:
compt_tree_size = [25, 20]

print("Creating test_loader...")
test_loader = LinkNeighborLoader(
    data=data,
    num_neighbors=compt_tree_size,
    neg_sampling_ratio=1,
    edge_label_index=("artist", "collab_with", "artist"),
    batch_size=128,
    num_workers=10,
    pin_memory=True,
)

Creating test_loader...


/home/aleferu/miniforge3/envs/musicbrainz/lib/python3.12/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


In [8]:
class MLPSAGEConv(MessagePassing):
    r"""The GraphSAGE operator with an MLP instead of the linear transformation.

    Args:
        in_channels (int or tuple): Size of each input sample, or :obj:`-1` to
            derive the size from the first input(s) to the forward method.
            A tuple corresponds to the sizes of source and target
            dimensionalities.
        out_channels (int): Size of each output sample.
        hidden_channels (int, optional): Size of the hidden layer in the MLP.
            (default: :obj:`64`)
        aggr (str or Aggregation, optional): The aggregation scheme to use.
            Any aggregation of :obj:`torch_geometric.nn.aggr` can be used,
            *e.g.*, :obj:`"mean"`, :obj:`"max"`, or :obj:`"lstm"`.
            (default: :obj:`"mean"`)
        normalize (bool, optional): If set to :obj:`True`, output features
            will be :math:`\ell_2`-normalized, *i.e.*,
            :math:`\frac{\mathbf{x}^{\prime}_i}
            {\| \mathbf{x}^{\prime}_i \|_2}`.
            (default: :obj:`False`)
        root_weight (bool, optional): If set to :obj:`False`, the layer will
            not add transformed root node features to the output.
            (default: :obj:`True`)
        bias (bool, optional): If set to :obj:`False`, the layer will not learn
            an additive bias. (default: :obj:`True`)
        **kwargs (optional): Additional arguments of
            :class:`torch_geometric.nn.conv.MessagePassing`.

    Shapes:
        - **inputs:**
          node features :math:`(|\mathcal{V}|, F_{in})` or
          :math:`((|\mathcal{V_s}|, F_{s}), (|\mathcal{V_t}|, F_{t}))`
          if bipartite,
          edge indices :math:`(2, |\mathcal{E}|)`
        - **outputs:** node features :math:`(|\mathcal{V}|, F_{out})` or
          :math:`(|\mathcal{V_t}|, F_{out})` if bipartite
    """

    def __init__(
        self,
        in_channels: Union[int, Tuple[int, int]],
        out_channels: int,
        hidden_channels: int = 64,
        aggr: Optional[Union[str, List[str], Aggregation]] = "mean",
        normalize: bool = False,
        root_weight: bool = True,
        bias: bool = True,
        **kwargs,
    ):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.hidden_channels = hidden_channels
        self.normalize = normalize
        self.root_weight = root_weight

        if isinstance(in_channels, int):
            in_channels = (in_channels, in_channels)

        if aggr == "lstm":
            kwargs.setdefault("aggr_kwargs", {})
            kwargs["aggr_kwargs"].setdefault("in_channels", in_channels[0])
            kwargs["aggr_kwargs"].setdefault("out_channels", in_channels[0])

        super().__init__(aggr, **kwargs)


        self.mlp = Sequential(
            Linear(in_channels[0], hidden_channels),
            ReLU(),
            Linear(hidden_channels, in_channels[0]) # Output size should match input for aggregation
        )

        if isinstance(self.aggr_module, MultiAggregation):
            aggr_out_channels = self.aggr_module.get_out_channels(in_channels[0])
        else:
            aggr_out_channels = in_channels[0]

        self.lin_l = Linear(aggr_out_channels, out_channels, bias=bias)
        if self.root_weight:
            self.lin_r = Linear(in_channels[1], out_channels, bias=False)

        self.reset_parameters()

    def reset_parameters(self):
        super().reset_parameters()
        for layer in self.mlp:
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()
        self.lin_l.reset_parameters()
        if self.root_weight:
            self.lin_r.reset_parameters()

    def forward(
        self,
        x: Union[Tensor, OptPairTensor],
        edge_index: Adj,
        size: Size = None,
    ) -> Tensor:

        if isinstance(x, Tensor):
            x = (x, x)

        # Propagate through MLP
        x = (self.mlp(x[0]), x[1])

        # propagate_type: (x: OptPairTensor)
        out = self.propagate(edge_index, x=x, size=size)
        out = self.lin_l(out)

        x_r = x[1]
        if self.root_weight and x_r is not None:
            out = out + self.lin_r(x_r)

        if self.normalize:
            out = F.normalize(out, p=2.0, dim=-1)

        return out

    def message(self, x_j: Tensor) -> Tensor:
        return x_j

    def message_and_aggregate(self, adj_t: Adj, x: OptPairTensor) -> Tensor:
        if isinstance(adj_t, SparseTensor):
            adj_t = adj_t.set_value(None, layout=None)
        return spmm(adj_t, x[0], reduce=self.aggr)

    def __repr__(self) -> str:
        return (
            f"{self.__class__.__name__}({self.in_channels}, "
            f"{self.out_channels}, hidden_channels={self.hidden_channels}, aggr={self.aggr})"
        )



In [9]:
class GNN(torch.nn.Module):
    def __init__(self, metadata, hidden_channels, out_channels):
        super().__init__()
        self.metadata = metadata
        self.out_channels = out_channels

        self.conv1 = HeteroConv({
            ("artist", "collab_with", "artist"): GATConv((artist_channels, artist_channels), hidden_channels),
            ("artist", "has_tag_artists", "tag"): SAGEConv((artist_channels, tag_channels), hidden_channels),
            # ("artist", "last_fm_match", "artist"): GATConv((artist_channels, artist_channels), hidden_channels),
            ("track", "has_tag_tracks", "tag"): SAGEConv((track_channels, tag_channels), hidden_channels),
            ("artist", "linked_to", "artist"): GATConv((artist_channels, artist_channels), hidden_channels),
            ("artist", "musically_related_to", "artist"): GATConv((artist_channels, artist_channels), hidden_channels),
            ("artist", "personally_related_to", "artist"): GATConv((artist_channels, artist_channels), hidden_channels),
            ("tag", "tags_artists", "artist"): SAGEConv((tag_channels, artist_channels), hidden_channels),
            ("tag", "tags_tracks", "track"): SAGEConv((tag_channels, track_channels), hidden_channels),
            ("track", "worked_by", "artist"): SAGEConv((track_channels, artist_channels), hidden_channels),
            ("artist", "worked_in", "track"): SAGEConv((artist_channels, track_channels), hidden_channels),
        }, aggr="mean")

        self.conv2 = HeteroConv({
            ("artist", "collab_with", "artist"): GATConv((hidden_channels, hidden_channels), hidden_channels),
            ("artist", "has_tag_artists", "tag"): SAGEConv((hidden_channels, hidden_channels), hidden_channels),
            # ("artist", "last_fm_match", "artist"): GATConv((hidden_channels, hidden_channels), hidden_channels),
            ("track", "has_tag_tracks", "tag"): SAGEConv((hidden_channels, hidden_channels), hidden_channels),
            ("artist", "linked_to", "artist"): GATConv((hidden_channels, hidden_channels), hidden_channels),
            ("artist", "musically_related_to", "artist"): GATConv((hidden_channels, hidden_channels), hidden_channels),
            ("artist", "personally_related_to", "artist"): GATConv((hidden_channels, hidden_channels), hidden_channels),
            ("tag", "tags_artists", "artist"): SAGEConv((hidden_channels, hidden_channels), hidden_channels),
            ("tag", "tags_tracks", "track"): SAGEConv((hidden_channels, hidden_channels), hidden_channels),
            ("track", "worked_by", "artist"): SAGEConv((hidden_channels, hidden_channels), hidden_channels),
            ("artist", "worked_in", "track"): SAGEConv((hidden_channels, hidden_channels), hidden_channels),
        }, aggr="mean")

        self.linear1 = Linear(hidden_channels * 2, hidden_channels * 4)
        self.linear2 = Linear(hidden_channels * 4, out_channels)

    def forward(self, x_dict, edge_index_dict):
        x_dict1 = self.conv1(x_dict, edge_index_dict)
        x_dict2 = self.conv2(x_dict1, edge_index_dict)

        x_artist = torch.cat([x_dict1['artist'], x_dict2['artist']], dim=-1)

        x_artist = self.linear1(x_artist)
        x_artist = self.linear2(x_artist)

        # Normalize the artist node features
        x_artist = F.normalize(x_artist, p=2, dim=-1)

        # Update the dictionary with the new 'artist' features, leaving other nodes unchanged
        x_dict['artist'] = x_artist

        return x_dict

In [10]:
model = GNN(metadata=data.metadata(), hidden_channels=64, out_channels=64).to(device)
model.load_state_dict(
    torch.load(model_path)
)
model.eval()

/tmp/ipykernel_83469/1260903973.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(model_path)


GNN(
  (conv1): HeteroConv(num_relations=10)
  (conv2): HeteroConv(num_relations=10)
  (linear1): Linear(128, 256, bias=True)
  (linear2): Linear(256, 64, bias=True)
)

In [11]:
def test_model(model, test_loader, criterion, device, threshold, train_edges_set):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    all_probs = []
    total_loss = 0.0
    valid_batches = 0

    with torch.no_grad():  # Disable gradient computation
        for sampled_data in tqdm.tqdm(test_loader):
            # Move data to the device
            sampled_data = sampled_data.to(device)

            # Forward pass
            pred_dict = model(sampled_data.x_dict, sampled_data.edge_index_dict)

            # Get predictions and labels for the 'collab_with' edge type
            edge_label_index = sampled_data['artist', 'collab_with', 'artist'].edge_label_index
            edge_label = sampled_data['artist', 'collab_with', 'artist'].edge_label

            # Filter edge list
            filtered_edges = []
            filtered_edge_label = []
            positive_count = 0
            for src, dst, label in zip(edge_label_index[0, :], edge_label_index[1, :], edge_label):
                lookup_edge = (
                    sampled_data['artist'].n_id[src].item(),
                    sampled_data['artist'].n_id[dst].item()
                )
                if lookup_edge in train_edges_set:
                    continue
                filtered_edges.append([src.item(), dst.item()])
                label_item = label.item()
                filtered_edge_label.append(label_item)

                # Balancing
                if np.isclose(label_item, 1):
                    positive_count += 1
                else:
                    positive_count -= 1
                    if positive_count == 0:
                        break

            if len(filtered_edges) == 0:
                continue  # Skip if no valid edges left

            valid_batches += 1

            # Normal evaluation with the rests
            filtered_edges = torch.tensor(filtered_edges, dtype=torch.long).t().to(device)
            filtered_labels = torch.tensor(filtered_edge_label).to(device)

            src_emb = pred_dict['artist'][filtered_edges[0]]  # Source node embeddings
            dst_emb = pred_dict['artist'][filtered_edges[1]]  # Destination node embeddings

            # Compute the dot product between source and destination embeddings
            preds = (src_emb * dst_emb).sum(dim=-1)  # Scalar for each edge
            probs = torch.sigmoid(preds)  # Convert logits to probabilities
            preds_binary = (probs > threshold).long()  # Convert probabilities to binary predictions

            loss = criterion(preds, filtered_labels.float())
            total_loss += loss

            # Collect predictions and labels
            all_preds.append(preds_binary.cpu())
            all_labels.append(filtered_labels.cpu())
            all_probs.append(probs.cpu())

    # Concatenate all predictions and labels
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    all_probs = torch.cat(all_probs)
        
    # Average loss
    total_loss /= valid_batches

    # Compute metrics
    cm = confusion_matrix(all_labels, all_preds)
    tp = cm[1, 1]
    fp = cm[0, 1]
    fn = cm[1, 0]
    tn = cm[0, 0]
    accuracy = (tp + tn) / (tp + fp + fn + tn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    roc_auc = roc_auc_score(all_labels, all_probs)

    print("Test Results:")
    print(f"Loss:      {total_loss:.4f}")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-score:  {f1:.4f}")
    print(f"ROC-AUC:   {roc_auc:.4f}")
    print(f"Confusion Matrix:\n{tp} {fn}\n{fp} {tn}")

In [12]:
test_model(
    model,
    test_loader, 
    F.binary_cross_entropy_with_logits,
    device,
    best_threshold,
    train_edges_set
)

100%|██████████| 19243/19243 [09:38<00:00, 33.28it/s]


Test Results:
Loss:      0.5565
Accuracy:  0.8379
Precision: 0.6352
Recall:    0.9338
F1-score:  0.7561
ROC-AUC:   0.9124
Confusion Matrix:
122892 8706
70570 286948
